### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E"> 🚘 Car valuation</p>

<a id="1.2"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Content</div>

<div class="toc"><ul class="toc-item"><li><span><a href="#Data-description" data-toc-modified-id="Data-description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data description</a></span></li><li><span><a href="#Loading-libraries" data-toc-modified-id="Loading-libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Loading libraries</a></span></li><li><span><a href="#EDA" data-toc-modified-id="EDA-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>EDA</a></span></li><li><span><a href="#Data_preprocessing" data-toc-modified-id="Data_preprocessing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data preprocessingх</a></span></li><li><span><a href="#Prepare-samples-for-training-models" data-toc-modified-id="Prepare-samples-for-training-models-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Prepare samples for training models</a></span></li><li><span><a href="#Direct-features-coding" data-toc-modified-id="Direct-features-coding-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Direct features coding</a></span></li><li><span><a href="#Standartization" data-toc-modified-id="Standartization-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Standartization</a></span></li><li><span><a href="#Model-training" data-toc-modified-id="Model-training-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Model training</a></span></li><li><span><a href="#Model-comparison" data-toc-modified-id="Model-comparison-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Model comparison</a></span></li><li><span><a href="#Testing-best-model" data-toc-modified-id="Testing-best-model-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Testing-best-model</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Conclusion</a></span></li>

<a id="Data-description"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Data description</div>

<ul>
<li>DateCrawled — date when the questionnaire was downloaded from the database</li>
<li>VehicleType - car body type</li>
<li>RegistrationYear - the year the car was registered</li>
<li>Gearbox - gear type</li>
<li>Power - power (hp)</li>
<li>Model - car model</li>
<li>Kilometer - mileage (km)</li>
<li>RegistrationMonth — car registration month</li>
<li>FuelType - type of fuel</li>
<li>Brand - car brand</li>
<li>Repaired - was the car under repair or not</li>
<li>DateCreated — date of creation of the questionnaire</li>
<li>NumberOfPictures - number of car photos</li>
<li>PostalCode - postal code of the owner of the profile (user)</li>
<li>LastSeen — date of last user activity</li>
</ul>


<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">



Service for selling used cars "Not beaten, not beautiful" is developing an application to attract new customers. It will show you the market value of your car.

</div>

<a id="Loading_libraries"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Loading libraries</div>

In [1]:
import os
import time
import warnings
import pandas as pd
import numpy as np
import catboost as cb
import lightgbm as lgmb
from sklearn import tree, metrics
from tabulate import tabulate
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split

<a id="EDA"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> EDA</div>

In [2]:
pth1 = '/datasets/autos.csv'
pth2 = 'autos.csv'
if os.path.exists(pth1):
    data = pd.read_csv(pth1)
elif os.path.exists(pth2):
    data = pd.read_csv(pth2)
else:
    print('Something is wrong')

In [3]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
💡 The same dates were found in the "DateCrawled" and "DateCreated" columns. It is necessary to check the data of the columns, if so, then delete one of them.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
There are gaps in the data in the "VehicleType", "Gearbox", "FuelType" and "Repaired" columns. Change the column type "DateCrawled" to datatype. Change the column type "RegistrationYear" and "RegistrationMonth" to int. The column "Model" has gaps, it is necessary to delete the rows with gaps. it is one of the key signs for forecasting.

In [5]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
 - The maximum year 9999 was found in the "RegistrationYear" column data, which is incorrect. It is necessary to examine this column and remove all rows with an incorrect year of registration.
     <br> - It follows from the data that the "NumberOfPictures" column consists of zeros. You need to check this column, if so, you need to delete it.
     <br> - The "PostalCode" column is not useful in predicting data. You can delete it.
     <br> - In the data column "Power" the maximum power is 20000, which according to the Internet is not reasonable. Because To date, TranStar Racing LLC's Dagger GT has been named the most powerful passenger car in the world, with an engine power exceeding 2000 horsepower (2028 hp).
    <br> - A zero price was found in the data of the "Price" column, apparently the seller did not indicate the price of the car in the questionnaire. But this column is a target feature, and it is important in data prediction, so we will remove columns with zero price. - The maximum year 9999 was found in the data of the "RegistrationYear" column, which is incorrect. It is necessary to examine this column and remove all rows with an incorrect year of registration.
     <br> - It follows from the data that the "NumberOfPictures" column consists of zeros. You need to check this column, if so, you need to delete it.
     <br> - The "PostalCode" column is not useful in predicting data. You can delete it.
     <br> - In the data column "Power" the maximum power is 20000, which according to the Internet is not reasonable. Because To date, TranStar Racing LLC's Dagger GT has been named the most powerful passenger car in the world, with an engine power exceeding 2000 horsepower (2028 hp).
    <br> - A zero price was found in the data of the "Price" column, apparently the seller did not indicate the price of the car in the questionnaire. But this column is a target feature, and it is important when predicting data, so we will remove the columns with a zero price.

In [6]:
#check for duplicates
data.duplicated().sum()

4

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
    The data contains duplicates, you need to remove them.

<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">
<b>✍ Conclusion</b>
<br>Duplicates and gaps found in data. You need to change the column type "DateCrawled". Incorrect column name detected. It is necessary to check the column "RegistrationYear" and "NumberOfPictures" for the correctness of the data.
        
</div>

<a id="Data_preprocessing"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Data preprocessing</div>

In [7]:
#delete duplicates
data = data.drop_duplicates()

In [8]:
#renames columns
data = data.rename(columns = {
    'DateCrawled': 'Date_crawled',
    'VehicleType': 'Vehicle_type',
    'RegistrationYear': 'Registration_year',
    'RegistrationMonth': 'Registration_month',
    'FuelType': 'Fuel_type',
    'DateCreated': 'Date_created',
    'NumberOfPictures': 'Number_of_pictures',
    'PostalCode': 'Postal_code',
    'LastSeen': 'Last_seen'})

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
Let's delete the "Date_crawled" column. date is similar to Date_created

In [9]:
data = data.drop(['Date_crawled'], axis = 1)

In [10]:
#change column type 'Date_created'
data['Date_created'] = pd.to_datetime(data['Date_created'], format="%Y-%m-%dT%H:%M:%S")

In [11]:
#how many zeros rows in "Number_of_pictures"
len(data[data['Number_of_pictures'] == 0])

354365

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
    All rows in the "Number_of_pictures" column are null, hence the need to remove the column.

In [12]:
data = data.drop(columns = ['Number_of_pictures'], axis = 1)

In [13]:
#change type columns "Registration_year" and "Registration_month"
data['Registration_year'] = data['Registration_year'].astype(int)
data['Registration_month'] = data['Registration_month'].astype(int)

In [14]:
#let's check column data "Registration_year"
data = data.loc[data['Registration_year'] < 2016]

In [15]:
data['Model'].isna().sum()

16351

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
    This is 5% of the data from the total. We can remove them.

In [16]:
#remove data with missing values in the "Model" column
data = data.loc[~data['Model'].isna()]

In [17]:
data = data.drop(columns = ['Postal_code'], axis = 1)

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
The "Power" column has many zero values and values greater than 800 hp, so let's replace them with median values.

In [18]:
#replace incorrect values with nan
data.loc[(data['Power'] > 800) | (data['Power'] == 0), 'Power'] = None 

In [19]:
#replace nan in the "Power" column with the median value
data['Power'] = data['Power'].fillna(data.groupby(['Model'])['Power'].transform('median'))

In [20]:
data = data.loc[~data['Power'].isna()]

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
In the "Vehicle_type", "Gearbox", "Fuel_type" columns, the missing values will be replaced by frequently occurring ones, and in the "Repaired" missing values will be replaced by 'unknown'.

In [21]:
data['Vehicle_type'] = data['Vehicle_type'].fillna(
    data.groupby('Model')['Vehicle_type'].transform(lambda x: x.value_counts().idxmax()))

In [22]:
data['Gearbox'] = data['Gearbox'].fillna(
    data.groupby('Model')['Gearbox'].transform(lambda x: x.value_counts().idxmax()))

In [23]:
data['Fuel_type'] = data['Fuel_type'].fillna(
    data.groupby('Model')['Fuel_type'].transform(lambda x: x.value_counts().idxmax()))

In [24]:
data['Repaired'] = data['Repaired'].fillna('unknown')

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
Columns "Date_created", "Last_seen" will be deleted, because are uninformative.

In [25]:
data = data.drop(['Date_created'], axis =1)
data = data.drop(['Last_seen'], axis =1)

In [26]:
#removed lines with zero price, they are clearly wrong
data = data.loc[data['Price'] != 0]

<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">

<b>✍ Conclusion</b>
         <ul>
<li>Duplicates removed.</li>
<li>Columns renamed.</li>
<li>Columns "VehicleType", "Gearbox", "FuelType" are filled with frequently occurring values.</li>
<li>The data in the "RegistrationYear", "Power", "Repaired" columns has been corrected.</li>
<li>Rows with missing values in the "Model" column have been removed.</li>
<li>Data type changed in "RegistrationMonth", "DateCreated" column.</li>
<li>Columns "NumberOfPictures", "PostalCode", "Date_crawled", "Date_created", "Last_seen" have been removed because are not informative.</li>
</ul>
</div>

<a id="Prepare-samples-for-training-models"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Prepare samples for training models</div>

In [27]:
X = data.drop('Price', axis=1)
y = data['Price']

In [28]:
X_train, X_test_1, y_train, y_test_1  = train_test_split(X, y, test_size=0.4, random_state=42)
X_valid, X_test, y_valid, y_test =train_test_split(X_test_1, y_test_1, test_size=0.5, random_state=42)

In [29]:
print(f"Training sample size: {X_train.shape}")

print(f"Validation sample size: {X_valid.shape}")

print(f"Testing sample size: {X_test.shape}")

Training sample size: (183834, 10)
Validation sample size: (61278, 10)
Testing sample size: (61278, 10)


<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">

<b>✍ Conclusion</b>
        <ul>
<li>Data splited on the training, validation and testing sample.</li>

</ul>
</div>

<a id="Direct-features-coding"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Direct features coding</div>

In [30]:
X_train_ohe = pd.get_dummies(X_train, drop_first=True)
X_valid_ohe = pd.get_dummies(X_valid, drop_first=True)
X_test_ohe = pd.get_dummies(X_test, drop_first=True)

In [31]:
X_train_ohe

,Registration_year,Power,Kilometer,Registration_month,Vehicle_type_convertible,Vehicle_type_coupe,Vehicle_type_other,Vehicle_type_sedan,Vehicle_type_small,Vehicle_type_suv,...,Brand_skoda,Brand_smart,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,Repaired_unknown,Repaired_yes
271146,2007,109.0,150000,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21506,1999,75.0,150000,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
33507,2006,218.0,150000,8,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
166662,2005,88.0,80000,5,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
192369,2008,95.0,125000,10,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138497,2008,170.0,150000,10,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
299841,2011,141.0,125000,8,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
152465,2002,231.0,150000,8,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
169770,2007,235.0,150000,7,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
X_valid_ohe.shape

(61278, 302)

In [33]:
X_test_ohe.shape

(61278, 304)

In [34]:
X_train_ohe.shape

(183834, 306)

In [35]:
def remove_mismatched_columns(train, valid, test):
    common_columns = set(train.columns) & set(valid.columns) & set(test.columns)

    train = train[common_columns]
    valid = valid[common_columns]
    test = test[common_columns]

    return train, valid, test

X_train_ohe, X_valid_ohe, X_test_ohe = remove_mismatched_columns(
    X_train_ohe, X_valid_ohe, X_test_ohe
)

/var/folders/cg/b2wsqr_s7fd0x08bc2ykkndc0000gn/T/ipykernel_3160/1622521156.py:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  train = train[common_columns]
/var/folders/cg/b2wsqr_s7fd0x08bc2ykkndc0000gn/T/ipykernel_3160/1622521156.py:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  valid = valid[common_columns]
/var/folders/cg/b2wsqr_s7fd0x08bc2ykkndc0000gn/T/ipykernel_3160/1622521156.py:6: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  test = test[common_columns]


In [36]:
X_valid_ohe.shape

(61278, 300)

In [37]:
X_test_ohe.shape

(61278, 300)

In [38]:
X_train_ohe.shape

(183834, 300)

In [39]:
encoder = OrdinalEncoder()
cat_columns =  ['Vehicle_type', 'Gearbox', 'Model', 'Fuel_type', 'Brand', 'Repaired']
encoder.fit(X_train[cat_columns])

X_train_ord = X_train.copy()
X_valid_ord = X_valid.copy()
X_test_ord = X_test.copy()

X_train_ord[cat_columns] = encoder.transform(X_train_ord[cat_columns])
X_valid_ord[cat_columns] = encoder.transform(X_valid_ord[cat_columns])
X_test_ord[cat_columns] = encoder.transform(X_test_ord[cat_columns])


<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">

<b>✍ Conclusion</b>
        <ul>
<li>For direct encoding, the pd.get_dummies() function was used. </li>
<li>For the ordinal coding used  function encoder.transform for columns 'Vehicle_type', 'Gearbox', 'Model', 'Fuel_type', 'Brand', 'Repaired'. </li>
</ul>
</div>

<a id="Standartization"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Standartization</div>

In [40]:
numeric_ord = ['Vehicle_type', 'Registration_year', 'Gearbox', 'Power', 'Model', 'Kilometer', 'Fuel_type', 'Brand','Registration_month', 'Repaired']
scaler = StandardScaler()
scaler.fit(X_train_ord[numeric_ord])
X_train_ord[numeric_ord] = scaler.transform(X_train_ord[numeric_ord])
X_valid_ord[numeric_ord] = scaler.transform(X_valid_ord[numeric_ord])
X_test_ord[numeric_ord] = scaler.transform(X_test_ord[numeric_ord])

In [41]:
numeric_ohe = ['Registration_year', 'Power', 'Kilometer', 'Registration_month']
scaler = StandardScaler()
scaler.fit(X_train_ohe[numeric_ohe])
X_train_ohe[numeric_ohe] = scaler.transform(X_train_ohe[numeric_ohe])
X_valid_ohe[numeric_ohe] = scaler.transform(X_valid_ohe[numeric_ohe])
X_test_ohe[numeric_ohe] = scaler.transform(X_test_ohe[numeric_ohe])

/var/folders/cg/b2wsqr_s7fd0x08bc2ykkndc0000gn/T/ipykernel_3160/2244036632.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_ohe[numeric_ohe] = scaler.transform(X_train_ohe[numeric_ohe])


<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">

<b>✍ Conclusion</b>
        <ul>
<li>The scale of numerical features was transformed by the standartization method. </li>

</ul>
</div>

<a id="Model-training"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Model training</div>

### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E">CatBoost</p>

In [42]:
#%%time

#parametrs_Cat = {'depth': [4,7,10],
#                 'learning_rate': [0.03, 0.1, 0.15],
#                 'iterations': [100, 200, 300]
#              }

#cb = cb.CatBoostRegressor(random_state=12345);

#grid_search = GridSearchCV(cb, 
#                           parametrs_Cat, 
#                           cv = 3,  
#                           n_jobs=-1);

#grid_search.fit(X_train, y_train);

#grid_search = GridSearchCV(cb, 
#                           parametrs_Cat, 
#                           cv = 3,  
#                           n_jobs=-1);

#grid_search.fit(X_train, y_train);

In [43]:
#grid_search.best_params_

In [44]:
#%%time
#pred = grid_search.best_estimator_.predict(X_valid)

In [45]:
#cat_features = ['Vehicle_type', 'Gearbox', 'Model', 'Fuel_type', 'Brand', 'Repaired']

In [46]:
start_time = time.perf_counter()
cb = CatBoostRegressor(random_state=12345, depth=10, iterations=300, learning_rate=0.15)
cb.fit(X_train_ohe, y_train)
time_cat_fit = float('{0:.2f}'.format(time.perf_counter() - start_time))

0:	learn: 4071.8536997	total: 76.1ms	remaining: 22.7s
1:	learn: 3666.8097939	total: 91ms	remaining: 13.6s
2:	learn: 3331.7521720	total: 106ms	remaining: 10.5s
3:	learn: 3062.1135815	total: 120ms	remaining: 8.9s
4:	learn: 2844.0449094	total: 134ms	remaining: 7.88s
5:	learn: 2664.4268821	total: 148ms	remaining: 7.25s
6:	learn: 2516.0948946	total: 162ms	remaining: 6.78s
7:	learn: 2397.1966065	total: 176ms	remaining: 6.41s
8:	learn: 2296.4861456	total: 190ms	remaining: 6.14s
9:	learn: 2210.4047529	total: 203ms	remaining: 5.9s
10:	learn: 2141.4138762	total: 217ms	remaining: 5.7s
11:	learn: 2086.4502235	total: 230ms	remaining: 5.53s
12:	learn: 2041.1276868	total: 244ms	remaining: 5.39s
13:	learn: 1996.0276256	total: 259ms	remaining: 5.28s
14:	learn: 1963.3309139	total: 272ms	remaining: 5.18s
15:	learn: 1935.1292316	total: 286ms	remaining: 5.08s
16:	learn: 1912.7337323	total: 302ms	remaining: 5.02s
17:	learn: 1893.6039228	total: 317ms	remaining: 4.96s
18:	learn: 1874.9810230	total: 332ms	rema

159:	learn: 1538.7228999	total: 2.59s	remaining: 2.27s
160:	learn: 1537.8256292	total: 2.61s	remaining: 2.25s
161:	learn: 1537.0103464	total: 2.63s	remaining: 2.24s
162:	learn: 1535.8493064	total: 2.65s	remaining: 2.23s
163:	learn: 1535.0445701	total: 2.67s	remaining: 2.21s
164:	learn: 1534.1674359	total: 2.69s	remaining: 2.2s
165:	learn: 1533.2109726	total: 2.71s	remaining: 2.19s
166:	learn: 1532.5240159	total: 2.73s	remaining: 2.17s
167:	learn: 1531.5767021	total: 2.74s	remaining: 2.16s
168:	learn: 1530.6774445	total: 2.76s	remaining: 2.14s
169:	learn: 1529.5323456	total: 2.78s	remaining: 2.12s
170:	learn: 1528.5714239	total: 2.79s	remaining: 2.11s
171:	learn: 1527.8730379	total: 2.81s	remaining: 2.09s
172:	learn: 1527.2616061	total: 2.83s	remaining: 2.07s
173:	learn: 1526.6619686	total: 2.84s	remaining: 2.06s
174:	learn: 1525.9452593	total: 2.86s	remaining: 2.04s
175:	learn: 1525.1761924	total: 2.87s	remaining: 2.02s
176:	learn: 1524.4643453	total: 2.89s	remaining: 2.01s
177:	learn:

In [47]:
start_time = time.perf_counter()
pred = cb.predict(X_valid_ohe)
time_cat_pred = float('{0:.2f}'.format(time.perf_counter() - start_time))

In [48]:
mse_cat = mean_squared_error(y_valid, pred)
rmse_cat = (mse_cat) ** 0.5
print('RMSE for the CatBoost model in the validation sample: ',rmse_cat)

RMSE for the CatBoost model in the validation sample:  1583.6787820501593


### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E">XGBoost</p>

In [49]:
start_time = time.perf_counter()
xgb = XGBRegressor(random_state=12345, learning_rate=0.1, max_depth=35)
xgb.fit(X_train_ohe, y_train)
time_xgb_fit = float('{0:.2f}'.format(time.perf_counter() - start_time))

In [50]:
start_time = time.perf_counter()
preds_xgb= xgb.predict(X_valid_ohe)
time_xgb_pred = float('{0:.2f}'.format(time.perf_counter() - start_time))

In [51]:
mse_xgb = mean_squared_error(y_valid, preds_xgb)
rmse_xgb = mse_xgb** 0.5
print('RMSE for XGBoost model in the validation sample: ', rmse_xgb)

RMSE for XGBoost model in the validation sample:  1682.6872503861384


### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E">LightGBM</p>

In [52]:
# %%time
# param_light = {'learning_rate': [0.1, 0.3, 0.5, 0.7],
#               'max_depth': [15, 20, 30, 35],
#              }

# lgbm = LGBMRegressor(random_state = 12345)

# # инициализируем GridSearchCV
# grid_search = GridSearchCV(estimator = lgbm, 
#                            param_grid = param_light, 
#                            cv = 3,
#                            n_jobs = -1
#                           );
# grid_search.fit(X_train_ord, y_train);

In [53]:
# grid_search.best_params_

In [54]:
# %%time
# preds_lgbm = grid_search.best_estimator_.predict(X_valid_ord)

In [55]:
start_time = time.perf_counter()
lgbm = LGBMRegressor(random_state = 12345, learning_rate=0.3, max_depth=15)
lgbm.fit(X_train_ohe, y_train);
time_lgbm_fit = float('{0:.2f}'.format(time.perf_counter() - start_time))

In [56]:
start_time = time.perf_counter()
preds_lgbm = lgbm.predict(X_valid_ohe)
time_lgbm_pred = float('{0:.2f}'.format(time.perf_counter() - start_time))

In [57]:
mse_lgbm = mean_squared_error(y_valid, preds_lgbm)
rmse_lgbm = mse_lgbm** 0.5
print('RMSE for LightGBM model in the validation sample: ', rmse_lgbm)

RMSE for LightGBM model in the validation sample:  1628.4221313312594


### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E">DecisionTree</p>

In [58]:
# %%time
# parametrs_DT = {'max_depth': [15, 35]}

# DT = DecisionTreeRegressor(random_state=12345)

# # инициализируем GridSearchCV
# grid_search = GridSearchCV(DT, 
#                            parametrs_DT, 
#                            scoring='neg_mean_squared_error', 
#                            cv=2
#                           )
# grid_search.fit(X_train_ord, y_train)


In [59]:
# grid_search.best_params_

In [60]:
# %%time
# preds_DT = grid_search.best_estimator_.predict(X_valid_ord)

In [61]:
start_time = time.perf_counter()
DT = DecisionTreeRegressor(random_state=12345, max_depth=15)
DT.fit(X_train_ord, y_train)
time_DT_fit = float('{0:.2f}'.format(time.perf_counter() - start_time))

In [62]:
start_time = time.perf_counter()
preds_DT = DT.predict(X_valid_ord)
time_DT_pred = float('{0:.2f}'.format(time.perf_counter() - start_time))

In [63]:
mse_DT = mean_squared_error(y_valid, preds_DT)
rmse_DT = (mse_DT) ** 0.5
print('RMSE for Decision Tree model in the validation sample: ',rmse_DT)

RMSE for Decision Tree model in the validation sample:  1907.9165283929392


### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E">RandomForest</p>

In [64]:
start_time = time.perf_counter()
RF = RandomForestRegressor(random_state=12345, n_estimators=80, max_depth = 15)
RF.fit(X_train_ord, y_train)
time_RF_fit = float('{0:.2f}'.format(time.perf_counter() - start_time))

In [65]:
start_time = time.perf_counter()
preds_RF = RF.predict(X_valid_ord)
time_RF_pred = float('{0:.2f}'.format(time.perf_counter() - start_time))

In [66]:
mse_RF = mean_squared_error(y_valid, preds_RF)
rmse_RF = (mse_RF) ** 0.5
print('RMSE for Random Forest model in the validation sample: ',rmse_RF)

RMSE for Random Forest model in the validation sample:  1642.4337038280435


### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E">LinearRegression</p>

In [67]:
start_time = time.perf_counter()
LR = LinearRegression()
LR.fit(X_train_ohe, y_train)
time_LR_fit = float('{0:.2f}'.format(time.perf_counter() - start_time))

In [68]:
start_time = time.perf_counter()
pred_LR = LR.predict(X_valid_ohe)
time_LR_pred = float('{0:.2f}'.format(time.perf_counter() - start_time))

In [69]:
mse_LR = mean_squared_error(y_valid, pred_LR)
rmse_LR = (mse_LR) ** 0.5
print('RMSE for Linear Regresion model in the validation sample: ', rmse_LR)

RMSE for Linear Regresion model in the validation sample:  2815.387022018015


<a id="Model-comparison"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Model comparison</div>

### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E">Evaluating the quality of models using the RMSE metric</p>

In [70]:
data_RMSE={'Models':['XGBoost','LightGBM','CatBoost', 'Decision Tree', 'Random Forest','Linear Regresion'],
     'RMSE': [rmse_xgb, rmse_lgbm, rmse_cat, rmse_DT, rmse_RF, rmse_LR]}
df_RMSE=pd.DataFrame(data_RMSE)
print(tabulate(df_RMSE, headers='keys', tablefmt='psql'))

+----+------------------+---------+
|    | Models           |    RMSE |
|----+------------------+---------|
|  0 | XGBoost          | 1682.69 |
|  1 | LightGBM         | 1628.42 |
|  2 | CatBoost         | 1583.68 |
|  3 | Decision Tree    | 1907.92 |
|  4 | Random Forest    | 1642.43 |
|  5 | Linear Regresion | 2815.39 |
+----+------------------+---------+


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
Lowest RMSE in CatBoost. Highest in Linear Regresion.

### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E">Анализ времени обучения</p>

In [71]:
data_fit={'Models':['XGBoost','LightGBM','CatBoost', 'Decision Tree', 'Random Forest','Linear Regresion'],
     'T_fit': [time_xgb_fit, time_lgbm_fit, time_cat_fit, time_DT_fit, time_RF_fit, time_LR_fit]}
df_fit=pd.DataFrame(data_fit)
print(tabulate(df_fit, headers='keys', tablefmt='psql'))

+----+------------------+---------+
|    | Models           |   T_fit |
|----+------------------+---------|
|  0 | XGBoost          |  424.32 |
|  1 | LightGBM         |    1.73 |
|  2 | CatBoost         |    5.09 |
|  3 | Decision Tree    |    0.39 |
|  4 | Random Forest    |   20.4  |
|  5 | Linear Regresion |    1.22 |
+----+------------------+---------+


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
    
XGBoost Gradient boosting model  takes a long time to study. 
Next model Random Forest, CatBoost and LightGBMgradient boosting models, decision tree model, linear regression model. 

### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E">Prediction Time Analysis</p>

In [72]:
data_pred={'Models':['XGBoost','LightGBM','CatBoost', 'Decision Tree', 'Random Forest','Linear Regresion'],
     'T_pred': [time_xgb_pred, time_lgbm_pred, time_cat_pred, time_DT_pred, time_RF_pred, time_LR_pred]}
df_pred=pd.DataFrame(data_pred)
print(tabulate(df_pred, headers='keys', tablefmt='psql'))

+----+------------------+----------+
|    | Models           |   T_pred |
|----+------------------+----------|
|  0 | XGBoost          |     1.09 |
|  1 | LightGBM         |     0.18 |
|  2 | CatBoost         |     0.04 |
|  3 | Decision Tree    |     0.01 |
|  4 | Random Forest    |     0.6  |
|  5 | Linear Regresion |     0.06 |
+----+------------------+----------+


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
    
The study formed the following order in descending order: XGBoost, Random Forest, LightGBM, CatBoost, Decision Tree,  Linear Regresion. Linear Regresion has fastest prediction.  


<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">

<b>✍ Conclusion</b>
         <ul>
<li>Research suggests that the best CatBoost model. </li>
<li>The second line of the best model is occupied by LightGBM. </li>
<li>Despite its fast learning rate, Linear Regression is the worst model.</li>
<li>XGBoost, Random Forest and Decision Tree performed well because their metric was less than 2500. </li>
</ul>
</div>

<a id="Testing-best-model"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Testing-best-model</div>

In [73]:
cb = CatBoostRegressor(random_state=12345, depth = 10,  iterations = 300, learning_rate = 0.15);

cb.fit(X_train_ord, y_train);

pred_final = cb.predict(X_test_ord)

0:	learn: 4077.6088471	total: 13.2ms	remaining: 3.94s
1:	learn: 3672.8266481	total: 26.8ms	remaining: 4s
2:	learn: 3336.3556557	total: 39.8ms	remaining: 3.94s
3:	learn: 3064.3504956	total: 51.6ms	remaining: 3.82s
4:	learn: 2844.2749244	total: 64.5ms	remaining: 3.81s
5:	learn: 2665.6099614	total: 76.9ms	remaining: 3.77s
6:	learn: 2514.8513264	total: 89.1ms	remaining: 3.73s
7:	learn: 2394.8211036	total: 102ms	remaining: 3.71s
8:	learn: 2293.0170369	total: 114ms	remaining: 3.68s
9:	learn: 2214.6386571	total: 127ms	remaining: 3.67s
10:	learn: 2152.2816142	total: 139ms	remaining: 3.65s
11:	learn: 2101.2465781	total: 151ms	remaining: 3.63s
12:	learn: 2046.4108575	total: 163ms	remaining: 3.59s
13:	learn: 2010.2955812	total: 175ms	remaining: 3.57s
14:	learn: 1979.7585000	total: 187ms	remaining: 3.55s
15:	learn: 1953.6271760	total: 199ms	remaining: 3.53s
16:	learn: 1922.7126232	total: 211ms	remaining: 3.51s
17:	learn: 1897.8534694	total: 223ms	remaining: 3.49s
18:	learn: 1882.8357715	total: 234

155:	learn: 1511.5694205	total: 1.99s	remaining: 1.84s
156:	learn: 1510.5427089	total: 2s	remaining: 1.82s
157:	learn: 1509.2739024	total: 2.02s	remaining: 1.81s
158:	learn: 1509.1125616	total: 2.03s	remaining: 1.8s
159:	learn: 1508.2108722	total: 2.04s	remaining: 1.79s
160:	learn: 1507.9164623	total: 2.06s	remaining: 1.77s
161:	learn: 1507.0297373	total: 2.07s	remaining: 1.76s
162:	learn: 1506.3285899	total: 2.08s	remaining: 1.75s
163:	learn: 1505.6519363	total: 2.09s	remaining: 1.74s
164:	learn: 1504.4215531	total: 2.1s	remaining: 1.72s
165:	learn: 1503.5092223	total: 2.12s	remaining: 1.71s
166:	learn: 1502.7378959	total: 2.13s	remaining: 1.7s
167:	learn: 1501.7678771	total: 2.14s	remaining: 1.68s
168:	learn: 1501.1177060	total: 2.16s	remaining: 1.67s
169:	learn: 1500.3039606	total: 2.17s	remaining: 1.66s
170:	learn: 1498.9816544	total: 2.18s	remaining: 1.65s
171:	learn: 1498.4157919	total: 2.19s	remaining: 1.63s
172:	learn: 1497.5576096	total: 2.21s	remaining: 1.62s
173:	learn: 1497

In [74]:
mse_final = mean_squared_error(y_test, pred_final)
rmse_final = (mse_final) ** 0.5
print('RMSE on testing sample: ', rmse_final)

RMSE on testing sample:  1585.4354639414657


<a id="Conclusion"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Conclusion</div>

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<br>📑A study was conducted to determine the <b> value of cars</b>.
        
        
<br><b> Input data - data on technical characteristics, equipment and prices of other cars. </b>

        
<br> 📊 The data was preprocessed as follows: duplicate rows were removed; column names corrected; the columns "VehicleType", "Gearbox", "FuelType" are filled with frequently occurring values; the data in the columns "RegistrationYear", "Power", "Repaired", "Model" has been corrected; data type changed in "RegistrationMonth", "DateCreated" column; columns "NumberOfPictures", "PostalCode", "Date_crawled", "Date_created", "Last_seen" have been removed because are uninformative.
 
<br> 📊 Scaled and standardized data.
        
<br> 📊 Linear regression, random forest, decision tree, XGBoost, CatBoost, LightGBM models were trained.
        
<br> 📊 Found the best hyperparameters for the respective models using GridSearchCV.
        
<br> 📊 A comparison of the quality of the studied models using the RMSE metric, as well as training and prediction times is shown.
 
<br>Based on the analysis, the conclusion is made:
<br> <b>- the best CatBoost model, then LightGBM, after Random Forest.</b>
<br> <b>CatBoost </b> is the fastest training and prediction model for gradient boosting compared to LightGBM and XGBoost.
<br> <b>is the worst Linear Regression</b> model, it has the highest RMSE metric.